In [1]:
import numpy as np
import os
import json
import pandas as pd
import time
from tqdm import tqdm
import gc
import mmap
from scipy import spatial
import heapq
import random

In [2]:
import import_ipynb

In [3]:
from src.show import show_playlist
from get_similar_tracks import SimilarTraks
from create_co_occour_mat import CoOcc
collection_dir = '/media/harsh/Common/IITD/COL764-IR/Project/spotify_million_playlist_dataset/data/'
#random.seed(0)

importing Jupyter notebook from get_similar_tracks.ipynb
Creating Track Vocabulary
Reading the dataset
for file       : mpd.slice.102000-102999.json
total playlists: 1000
for file       : mpd.slice.103000-103999.json
total playlists: 1000
for file       : mpd.slice.104000-104999.json
total playlists: 1000
for file       : mpd.slice.101000-101999.json
total playlists: 1000
for file       : mpd.slice.10000-10999.json
total playlists: 1000
total time taken 0.8872902393341064
950
Total similar playlists 50
total tracks: 569
Total similar playlists 50
total tracks: 4460
importing Jupyter notebook from create_co_occour_mat.ipynb
Creating Track Vocabulary
Creating Track Vocabulary


In [4]:
class Rerank:
    def __init__(self, coll_dir, TVocab, idf, select_subset_of = None, apply_pca = False):
        self.coll_dir = coll_dir
        self.vocab = TVocab
        self.idf = idf
        self.select_subset_of = select_subset_of
        self.co_occ_handler = CoOcc(collection_dir=self.coll_dir, TVocab=self.vocab, idf=self.idf)
        self.sim_tracks = SimilarTraks(TVocab=TVocab,idf=idf,collection_dir=self.coll_dir, top_k= 100, apply_pca=apply_pca)
    
    def get_unranked_tracks(self, pid):
        sim = self.sim_tracks
        recommended = sim.get_songs_for_p(pid, self.select_subset_of)
        print('reccommended len (finally 500)', len(recommended))
        if len(recommended) < 500:
            tracks_from_archive = []
            with open('tracks_archive.txt','r') as f:
                tracks_from_archive = f.read().split()
                #print('archieved tracks',len(tracks_from_archive))
            index = 0
            while(len(recommended)<500):
                recommended.append(tracks_from_archive[index])
                index+=1
        return recommended
    
    def get_relevent_tracks(self, pid):
        actual_tracks = []
        select_subset_of = self.select_subset_of
        playlist = show_playlist(self.coll_dir, pid, print_op=False)
        if playlist is None:
            return []
        for track in playlist['tracks']:
            actual_tracks.append(track['track_uri'])
        print('relevent tracks size:', len(actual_tracks))
        if select_subset_of is not None and select_subset_of < len(actual_tracks):
            random.seed(0)
            actual_tracks = random.sample(actual_tracks, select_subset_of)
        print('sampled relevent tracks',len(actual_tracks))
        return actual_tracks
    
    def get_reranked_tracks(self, pid):
        recommended = self.get_unranked_tracks(pid)
        #print('recommended tracks', len(recommended))
        #select subset of = 5 => get only random 5 songs from whole playlist
        relevent_subset = self.get_relevent_tracks(pid)
        #print('relevent track:',relevent_subset)
        sim_scores = np.zeros(len(recommended))
        co_occ_handler = self.co_occ_handler
        for rel_track in relevent_subset:
            for i, rec_track in enumerate(recommended):
                #print(co_occ_handler.TVocab.to_index(rel_track))
                #print(co_occ_handler.TVocab.to_index(rec_track))
                
                sim = co_occ_handler.get_sim_track_ij(rel_track, rec_track)
                #print('sim', sim)
                sim_scores[i] += sim
        ranked_index = np.argsort(sim_scores)[::-1]
        #print('%-20s'%'un sorted sim score', sim_scores[:10])
        #print('%-20s'%'sim scores',sim_scores[ranked_index][:10])
        ranked = []
        for i in ranked_index:
            ranked.append(recommended[i])
        ranked = ranked[:500]
        #print('%-20s'%'top reranked tracks', ranked[:10])
        #print('%-20s'%'no reranked tracks', len(ranked))
        return ranked
    

In [5]:
rerank = Rerank(collection_dir, TVocab= None, idf= None, select_subset_of=20, apply_pca=True)

Creating Track Vocabulary
Creating Track Vocabulary


In [10]:
%%time
_ = rerank.get_reranked_tracks(pid=324003)

Total similar playlists 100
total tracks: 2855
reccommended len (finally 500) 2855
relevent tracks size: 93
sampled relevent tracks 20
CPU times: user 9.5 s, sys: 204 ms, total: 9.7 s
Wall time: 9.7 s


In [11]:
%%time
v_pids =[]
with open('validation_pids_1.txt','r') as f:
    v_pids = f.read().split()
    v_pids = [int(pid) for pid in v_pids]
for pid in v_pids:
    rerank.get_reranked_tracks(pid)

Total similar playlists 100
total tracks: 3396
reccommended len (finally 500) 3396
relevent tracks size: 44
sampled relevent tracks 20
Total similar playlists 100
total tracks: 3404
reccommended len (finally 500) 3404
relevent tracks size: 86
sampled relevent tracks 20
Total similar playlists 100
total tracks: 6239
reccommended len (finally 500) 6239
relevent tracks size: 39
sampled relevent tracks 20
Total similar playlists 100
total tracks: 2703
reccommended len (finally 500) 2703
relevent tracks size: 131
sampled relevent tracks 20
Total similar playlists 100
total tracks: 7728
reccommended len (finally 500) 7728
relevent tracks size: 62
sampled relevent tracks 20
Total similar playlists 100
total tracks: 1569
reccommended len (finally 500) 1569
relevent tracks size: 38
sampled relevent tracks 20
Total similar playlists 100
total tracks: 6187
reccommended len (finally 500) 6187
relevent tracks size: 104
sampled relevent tracks 20
Total similar playlists 100
total tracks: 3409
reccom